In [1]:
import os
import numpy as np
import lxml
from lxml import objectify, etree, html
import json
import urllib3
from io import StringIO, BytesIO
import newspaper
from newspaper import Article
import pandas as pd

## XML

In [2]:
xmlstr = open('books.xml').read()

In [3]:
print(xmlstr)

<?xml version="1.0"?>
<catalog>
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
      <price>44.95</price>
      <publish_date>2000-10-01</publish_date>
      <description>An in-depth look at creating applications 
      with XML.</description>
   </book>
   <book id="bk102">
      <author>Ralls, Kim</author>
      <title>Midnight Rain</title>
      <genre>Fantasy</genre>
      <price>5.95</price>
      <publish_date>2000-12-16</publish_date>
      <description>A former architect battles corporate zombies, 
      an evil sorceress, and her own childhood to become queen 
      of the world.</description>
   </book>
   <book id="bk103">
      <author>Corets, Eva</author>
      <title>Maeve Ascendant</title>
      <genre>Fantasy</genre>
      <price>5.95</price>
      <publish_date>2000-11-17</publish_date>
      <description>After the collapse of a nanotechnology 
      society in England, the young

In [4]:
root = etree.fromstring(xmlstr)

print(root)

<Element catalog at 0x7f21f05a3bc8>


In [5]:
catalog = []

for element_lvl1 in root:
    
    book = {}
    for element_lvl2 in element_lvl1:
        txt = element_lvl2.text
        book[element_lvl2.tag] = '' if txt is None else txt
        print('{}: {}'.format(element_lvl2.tag, book[element_lvl2.tag]))

    catalog.append(book)
        
    print()

author: Gambardella, Matthew
title: XML Developer's Guide
genre: Computer
price: 44.95
publish_date: 2000-10-01
description: An in-depth look at creating applications 
      with XML.

author: Ralls, Kim
title: Midnight Rain
genre: Fantasy
price: 5.95
publish_date: 2000-12-16
description: A former architect battles corporate zombies, 
      an evil sorceress, and her own childhood to become queen 
      of the world.

author: Corets, Eva
title: Maeve Ascendant
genre: Fantasy
price: 5.95
publish_date: 2000-11-17
description: After the collapse of a nanotechnology 
      society in England, the young survivors lay the 
      foundation for a new society.

author: Corets, Eva
title: Oberon's Legacy
genre: Fantasy
price: 5.95
publish_date: 2001-03-10
description: In post-apocalypse England, the mysterious 
      agent known only as Oberon helps to create a new life 
      for the inhabitants of London. Sequel to Maeve 
      Ascendant.

author: Corets, Eva
title: The Sundered Grail
genre: 

In [6]:
catalog

[{'author': 'Gambardella, Matthew',
  'description': 'An in-depth look at creating applications \n      with XML.',
  'genre': 'Computer',
  'price': '44.95',
  'publish_date': '2000-10-01',
  'title': "XML Developer's Guide"},
 {'author': 'Ralls, Kim',
  'description': 'A former architect battles corporate zombies, \n      an evil sorceress, and her own childhood to become queen \n      of the world.',
  'genre': 'Fantasy',
  'price': '5.95',
  'publish_date': '2000-12-16',
  'title': 'Midnight Rain'},
 {'author': 'Corets, Eva',
  'description': 'After the collapse of a nanotechnology \n      society in England, the young survivors lay the \n      foundation for a new society.',
  'genre': 'Fantasy',
  'price': '5.95',
  'publish_date': '2000-11-17',
  'title': 'Maeve Ascendant'},
 {'author': 'Corets, Eva',
  'description': 'In post-apocalypse England, the mysterious \n      agent known only as Oberon helps to create a new life \n      for the inhabitants of London. Sequel to Maeve \n

## JSON

In [8]:
el = json.load(open('example.json', 'r'))

print(el, type(el))

{'joints': {'1': [813.75, 573.75, 1, 1, 0.423828125, 0.53125], '0': [783.75, 528.75, 1, 0, 0.408203125, 0.4895833333333333], '8': [806.25, 693.75, 1, 8, 0.419921875, 0.6423611111111112], '13': [753.75, 843.75, 1, 13, 0.392578125, 0.78125], '15': [791.25, 528.75, 1, 15, 0.412109375, 0.4895833333333333], '5': [806.25, 581.25, 1, 5, 0.419921875, 0.5381944444444444], '12': [768.75, 768.75, 1, 12, 0.400390625, 0.7118055555555556], '17': [813.75, 528.75, 1, 17, 0.423828125, 0.4895833333333333], '3': [828.75, 641.25, 1, 3, 0.431640625, 0.59375], '7': [798.75, 693.75, 1, 7, 0.416015625, 0.6423611111111112], '9': [776.25, 768.75, 1, 9, 0.404296875, 0.7118055555555556], '2': [821.25, 566.25, 1, 2, 0.427734375, 0.5243055555555556], '11': [791.25, 693.75, 1, 11, 0.412109375, 0.6423611111111112], '10': [761.25, 851.25, 1, 10, 0.396484375, 0.7881944444444444], '6': [791.25, 641.25, 1, 6, 0.412109375, 0.59375]}, 'rect': [700.0311468932622, 455.3073043146713, 867.4688531067378, 889.9688531067378]} <cl

## Parsing

### lxml

In [9]:
URL = 'https://www.bbc.com/news/uk-politics-46868194'

In [10]:
http = urllib3.PoolManager()
r = http.request('GET', URL) # HTTPResponse (status, data, and header)
parser = etree.HTMLParser()

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [11]:
r.data

b'\n\n<!DOCTYPE html>\n<html lang="en" id="responsive-news">\n<head  prefix="og: http://ogp.me/ns#">\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=1">\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <title>Brexit: MPs vote on Theresa May\'s deal - BBC News</title>\n    <meta name="description" content="Theresa May is expected to be defeated by MPs voting on the terms of her plan to leave the EU.">\n    <link rel="preload" as="style" href="https://static.bbc.co.uk/news/1.270.03189/stylesheets/services/news/compact.css" media="(max-width: 599px)">\n    <link rel="preload" as="style" href="https://static.bbc.co.uk/news/1.270.03189/stylesheets/services/news/tablet.css" media="(min-width: 600px) and (max-width: 1007px)">\n    <link rel="preload" as="style" href="https://static.bbc.co.uk/news/1.270.03189/stylesheets/services/news/wide.css" media="(min-width: 1008px)">\n\n            <script> window.

In [12]:
r.data.decode('utf8')

'\n\n<!DOCTYPE html>\n<html lang="en" id="responsive-news">\n<head  prefix="og: http://ogp.me/ns#">\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=1">\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <title>Brexit: MPs vote on Theresa May\'s deal - BBC News</title>\n    <meta name="description" content="Theresa May is expected to be defeated by MPs voting on the terms of her plan to leave the EU.">\n    <link rel="preload" as="style" href="https://static.bbc.co.uk/news/1.270.03189/stylesheets/services/news/compact.css" media="(max-width: 599px)">\n    <link rel="preload" as="style" href="https://static.bbc.co.uk/news/1.270.03189/stylesheets/services/news/tablet.css" media="(min-width: 600px) and (max-width: 1007px)">\n    <link rel="preload" as="style" href="https://static.bbc.co.uk/news/1.270.03189/stylesheets/services/news/wide.css" media="(min-width: 1008px)">\n\n            <script> window.o

In [13]:
page = etree.parse(BytesIO(r.data), parser)

In [14]:
title = page.xpath("//h1[@class='story-body__h1']")[0].text

print(title)

Brexit: MPs vote on Theresa May's deal


In [15]:
full_text = page.xpath("//div[@class='story-body__inner']")[0]

In [16]:
prestory = full_text[1].text

print(prestory)

MPs have begun to vote on Theresa May's Brexit deal for leaving the European Union. 


In [17]:
text = ''

for el in full_text[2:]:
    if el.tag in ['p']:
        text += el.text
        for subel in el:
            if subel.tag == 'a':
                text += subel.text
        text += '\n'
    
print(text)

The so-called "meaningful vote" comes at the end of five days of debate over the plan.
Mrs May called for politicians to back her deal or risk "letting the British people down".
But with many of her own MPs expected to join opposition parties to vote against the deal, it is widely expected to be defeated.
Labour, the SNP and Conservative MP Sir Edward Leigh all pulled the amendments that they had proposed to the deal before they came to a vote. 
Tory MP John Baron insisted MPs vote on his - which would have given the UK the right to terminate the Northern Ireland backstop without the agreement of the EU - but it was defeated by 600 votes to 24.
Closing the debate on her deal, Mrs May said the responsibility of MPs was "profound" and that her deal "delivers on the core tenets of Brexit... in a way that protects jobs, ensures our security and honours the integrity of our United Kingdom".
But Labour leader Jeremy Corbyn said backing the "botched and damaging" deal would be a "reckless lea

### newspaper

In [2]:
baseurls = ['https://tass.ru/obschestvo/']

In [6]:
n = 1

def get_topic(url):
    texts = []
    for i in range(6006960, 9999999):
        if len(texts) < n:
            article = Article(url + str(i))
            try:
                article.download()
                article.parse()
                if article.text != '':
                    texts.append(article.text)
            except:
                pass
    return texts

In [ ]:
df = pd.DataFrame()

for url in baseurls:
    texts = get_topic(url)
    df[os.path.basename(url)] = texts

Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006960 on URL https://tass.ru/obschestvo/6006960
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006961 on URL https://tass.ru/obschestvo/6006961
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006962 on URL https://tass.ru/obschestvo/6006962
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006963 on URL https://tass.ru/obschestvo/6006963
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006964 on URL https://tass.ru/obschestvo/6006964
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006965 on URL https://tass.ru/obschestvo/6006965
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6006966 on URL https:

Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007021 on URL https://tass.ru/obschestvo/6007021
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007022 on URL https://tass.ru/obschestvo/6007022
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007024 on URL https://tass.ru/obschestvo/6007024
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007025 on URL https://tass.ru/obschestvo/6007025
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007026 on URL https://tass.ru/obschestvo/6007026
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007027 on URL https://tass.ru/obschestvo/6007027
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007028 on URL https:

Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007081 on URL https://tass.ru/obschestvo/6007081
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007082 on URL https://tass.ru/obschestvo/6007082
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007083 on URL https://tass.ru/obschestvo/6007083
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007084 on URL https://tass.ru/obschestvo/6007084
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007085 on URL https://tass.ru/obschestvo/6007085
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007086 on URL https://tass.ru/obschestvo/6007086
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007087 on URL https:

Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007140 on URL https://tass.ru/obschestvo/6007140
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007141 on URL https://tass.ru/obschestvo/6007141
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007142 on URL https://tass.ru/obschestvo/6007142
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007143 on URL https://tass.ru/obschestvo/6007143
You must `download()` an article first!
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007145 on URL https://tass.ru/obschestvo/6007145
Article `download()` failed with 404 Client Error: Not Found for url: https://tass.ru/obschestvo/6007146 on URL https://tass.ru/obschestvo/6007146
Article `download()` failed with 404 Client Error: Not Found for url: https://

In [5]:
df

,
0,


In [ ]:
article = Article('https://tass.ru/obschestvo/6006970')